In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
from scipy.special import i0

In [2]:
class Message_Packet:
    def __init__(self, source: str, destination: str, message: str):
        self.source = source
        self.destination = destination
        self.message = message

    def __str__(self):
        return f"Source: {self.source}\nDestination: {self.destination}\nMessage: {self.message}"

# Example usage:
if __name__ == "__main__":
    # Creating an instance of Message class
    msg = Message_Packet("John", "Alice", "Hello, how are you?")
    print(msg)

Source: John
Destination: Alice
Message: Hello, how are you?


In [4]:
class Graph:
    def __init__(self):
        self.adj = {}
    
    def add_node(self, node):
        if node not in self.adj:
            self.adj[node] = []

    def add_edge(self, node1, node2):
        self.add_node(node1)
        self.add_node(node2)
        self.adj[node1].append(node2)
        self.adj[node2].append(node1)
        
    def __str__(self):
        return f"Nodes: {list(self.adj.keys())}"

class Message:
    def __init__(self, source: str, destination: str, message: str):
        self.source = source
        self.destination = destination
        self.message = message

    def __str__(self):
        return f"Source: {self.source}\nDestination: {self.destination}\nMessage: {self.message}"

# Example usage:
if __name__ == "__main__":
    graph = Graph()
    graph.add_edge('A', 'B')
    graph.add_edge('B', 'C')
    graph.add_edge('C', 'D')
    graph.add_edge('D', 'A')

    msg = Message('A', 'B', 'Hello, how are you?')
    print(msg)


Source: A
Destination: B
Message: Hello, how are you?
